In [1]:
import pandas as pd

# Load dataset
dataset_path = 'dataset.csv'
dataset = pd.read_csv(dataset_path)

# Load example rewrites
example_rewrites_path = 'example_rewrites.csv'
example_rewrites = pd.read_csv(example_rewrites_path)

# Display the first few rows of each dataset
dataset.head(), example_rewrites.head()

(   post_id                                              issue  \
 0        1          Is the school uniform a good or bad idea:   
 1        2                      Firefox vs internet explorer:   
 2        4  If your spouse committed murder and he or she ...   
 3        5                           Tv is better than books:   
 4        6                           Tv is better than books:   
 
                                            post_text  Inappropriateness  \
 0  people cant be forced to wear school uniforms,...                  1   
 1  That form of argument degrades this forum, and...                  1   
 2  No I wouldn't turn in my spouse. Just because ...                  1   
 3  TV is terrible. Except for Spongebob maybe. Th...                  1   
 4  I love books, I love getting lost in them. The...                  1   
 
    Toxic Emotions  Excessive Intensity  Emotional Deception  \
 0               0                    0                    0   
 1              

In [2]:
dataset.head(2)

,post_id,issue,post_text,Inappropriateness,Toxic Emotions,Excessive Intensity,Emotional Deception,Missing Commitment,Missing Seriousness,Missing Openness,Missing Intelligibility,Unclear Meaning,Missing Relevance,Confusing Reasoning,Other Reasons,Detrimental Orthography,Reason Unclassified,fold0.0
0,1,Is the school uniform a good or bad idea:,"people cant be forced to wear school uniforms,...",1,0,0,0,1,0,1,1,0,0,1,1,0,1,TRAIN
1,2,Firefox vs internet explorer:,"That form of argument degrades this forum, and...",1,1,0,1,1,1,1,1,1,1,0,0,0,0,TRAIN


In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load pre-trained model and tokenizer
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

def calculate_perplexity(text):
    inputs = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs['input_ids'])
        loss = outputs.loss
    return torch.exp(loss).item()

# Example usage
example_text = dataset['post_text'].iloc[0]
calculate_perplexity(example_text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

85.42070007324219

In [5]:
!pip install transformers
from transformers import BertModel, BertTokenizer

# Load pre-trained model and tokenizer
bert_model_name = 'bert-base-uncased'
bert_model = BertModel.from_pretrained(bert_model_name)
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)

# ... rest of your code ...

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [12]:
# Load pre-trained model and tokenizer
bert_model_name = 'bert-base-uncased'
bert_model = BertModel.from_pretrained(bert_model_name)
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)
from sklearn.metrics.pairwise import cosine_similarity

def get_embedding(text):
    inputs = bert_tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

def calculate_semantic_similarity(text1, text2):
    embedding1 = get_embedding(text1)
    embedding2 = get_embedding(text2)
    return cosine_similarity([embedding1], [embedding2])[0][0]

# Example usage
# Example usage with the first annotator's rewrite
example_text_1 = dataset['post_text'].iloc[0]
example_text_2 = example_rewrites['Rewritten more appropriate argument (Annotator 1)'].iloc[0]
print(f"Semantic similarity between original and improved text: {calculate_semantic_similarity(example_text_1, example_text_2)}")

Semantic similarity between original and improved text: 0.7751972675323486
